In [5]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [2]:
df = pd.read_csv("dataset.csv")  

In [3]:
window_size = 200  # number of samples per window (~2–4 seconds depending on frequency)
step_size = 100    # 50% overlap

X = []
y = []

for activity in df['activity'].unique():
    df_act = df[df['activity'] == activity]
    signals = df_act[['x-axis','y-axis','z-axis']].values
    
    for start in range(0, len(signals) - window_size, step_size):
        end = start + window_size
        X.append(signals[start:end])  # shape: (window_size, 3)
        y.append(activity)
        
X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)  # (num_windows, window_size, 3)
print("y shape:", y.shape)


X shape: (10726, 200, 3)
y shape: (10726,)


In [4]:
scaler = StandardScaler()
X_reshaped = X.reshape(-1, 3)
X_scaled = scaler.fit_transform(X_reshaped)
X_scaled = X_scaled.reshape(X.shape)

In [6]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)
num_classes = y_categorical.shape[1]

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(window_size, 3)),
    BatchNormalization(),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    
    Conv1D(128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 198, 64)           640       
                                                                 
 batch_normalization (BatchN  (None, 198, 64)          256       
 ormalization)                                                   
                                                                 
 conv1d_1 (Conv1D)           (None, 196, 64)           12352     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 98, 64)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 98, 64)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 96, 128)           2

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.3, random_state=42, stratify=y_encoded)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=64
)

Epoch 1/20
118/118 [==============================] - 7s 49ms/step - loss: 0.6876 - accuracy: 0.7788 - val_loss: 10.4773 - val_accuracy: 0.1085
Epoch 2/20
118/118 [==============================] - 6s 49ms/step - loss: 0.3338 - accuracy: 0.8811 - val_loss: 11.7857 - val_accuracy: 0.2191
Epoch 3/20
118/118 [==============================] - 6s 47ms/step - loss: 0.2355 - accuracy: 0.9150 - val_loss: 5.2923 - val_accuracy: 0.4664
Epoch 4/20
118/118 [==============================] - 6s 47ms/step - loss: 0.1675 - accuracy: 0.9403 - val_loss: 0.7850 - val_accuracy: 0.8424
Epoch 5/20
118/118 [==============================] - 6s 47ms/step - loss: 0.1258 - accuracy: 0.9580 - val_loss: 0.5574 - val_accuracy: 0.8620
Epoch 6/20
118/118 [==============================] - 6s 47ms/step - loss: 0.0858 - accuracy: 0.9691 - val_loss: 0.4112 - val_accuracy: 0.8978
Epoch 7/20
118/118 [==============================] - 6s 49ms/step - loss: 0.0950 - accuracy: 0.9679 - val_loss: 0.3101 - val_accuracy: 0.90

In [14]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

101/101 [==============================] - 0s 4ms/step - loss: 0.1230 - accuracy: 0.9671
Test Accuracy: 0.9671
